In [1]:
import numpy as np
import numpy as np
import pandas as pd
#import WindFarmGenetic_v1 # wind farm layout optimization using genetic algorithms classes
from datetime import datetime
import os
from sklearn.svm import SVR
import pickle


In [2]:
import os
os.getcwd()

'/home/naveen/arena/hackathon/shell/hackathon/python/Ver1_N'

In [3]:
##
wt_N=50  #number of turbines
NA_loc=[]  #index of obstructed locations
population_size=100  #population size/number of layouts for genetic algorithms
n_mc_samples=10000 #population/layout for SVM training
iteration_times=100  #number of iterations

# wind farm size, cells
cols_cells = 80  # number of cells each row
rows_cells = 80  # number of cells each column
cell_width = 50 # unit (TBD)--> have to be computed.
constraint_dist = 400 #unit distance between two turbines
constraint_perim = 50

radius=np.ceil(constraint_dist/cell_width)

#genetic algorithmic parameters.
elite_rate = 0.2 
cross_rate = 0.6
mutate_rate = 0.1

random_rate = 0.5
num_runs=2 #number of independent runs with random initialization.


In [4]:
perim_cells=np.ceil(constraint_perim/cell_width) #TBU generalize it for multiple boundary cells.
NA_loc=[] 
NA_loc.extend([i for i in range(cols_cells)]) #upper row
NA_loc.extend([i*(cols_cells) for i in range(rows_cells)]) #left boundary
NA_loc.extend([i*(cols_cells)+cols_cells-1 for i in range(rows_cells)])#right Boundary
NA_loc.extend([(cols_cells)*(rows_cells-1)+i for i in range(rows_cells)])#right Boundary


In [5]:
data_folder = "naveen_data" #temporary code related files. (MOdify it as per your specification)
svr_model_filename = 'svr_1s1d_N_13.svr' #svr training file.
if not os.path.exists(data_folder):
    os.makedirs(data_folder)

wds_data_folder = "{}/wds".format(data_folder)
if not os.path.exists(wds_data_folder):
    os.makedirs(wds_data_folder)
    
# initial populations saved folder
init_pops_data_folder = "{}/init_data".format(data_folder)
if not os.path.exists(init_pops_data_folder):
    os.makedirs(init_pops_data_folder)

#result folder
results_data_folder = data_folder+"/results"
if not os.path.exists(results_data_folder):
    os.makedirs(results_data_folder)
    
#result folder for sugga genetic algorithm
sg_result_folder = "{}/sg".format(results_data_folder)
if not os.path.exists(sg_result_folder):
    os.makedirs(sg_result_folder)

In [6]:
import WindFarmGenetic_v1 as WindFarmGenetic
wfg = WindFarmGenetic.WindFarmGenetic(rows=rows_cells, cols=cols_cells, N=wt_N, NA_loc=NA_loc, pop_size=population_size,
                                      iteration=iteration_times,cell_width=cell_width, elite_rate=elite_rate,
                                             cross_rate=cross_rate, random_rate=random_rate, mutate_rate=mutate_rate,constraint_dist=constraint_dist)

In [7]:
for i in range(num_runs):
    wfg.gen_init_pop_NA()
    wfg.save_init_pop_NA("{}/init_{}.dat".format(init_pops_data_folder, i),
                         "{}/init_{}_NA.dat".format(init_pops_data_folder, i))

Generation layout: 100%|██████████| 10/10 [00:06<00:00,  1.44it/s]


In [8]:
##Generate initial layouts: single layout--> 1d array of size row*col, layouts are set of num_training_sample_SVM different(random) layouts.  
train_mc_layouts, train_mc_layouts_NA = WindFarmGenetic.LayoutGridMCGenerator.gen_mc_grid_with_NA_loc_circle(rows_cells,
                                                                                                      cols_cells,
                                                                                                      n_mc_samples,
                                                                                                      wt_N, NA_loc,radius,
                                                                                                      "{}/mc_layout.dat".format(wds_data_folder),
                                                                                                      "{}/mc_layout_NA.dat".format(wds_data_folder))

layouts = np.genfromtxt("{}/mc_layout.dat".format(wds_data_folder), delimiter="  ", dtype=np.int32)

Generation layout: 100%|██████████| 100/100 [00:35<00:00,  2.81it/s]


In [9]:

wfg.mc_gen_xy_NA_hackathon(rows=rows_cells, cols=cols_cells, layouts=layouts, n=n_mc_samples, N=wt_N,  xfname="{}/x.dat".format(wds_data_folder),
                 yfname="{}/y.dat".format(wds_data_folder))

layouts: 100%|██████████| 100/100 [00:23<00:00,  4.30it/s]


In [9]:
x_original = pd.read_csv("{}/x.dat".format(wds_data_folder), header=None, nrows=rows_cells * cols_cells,
                         delim_whitespace=True, dtype=np.float32)
x_original = x_original.values

In [10]:
y_original = pd.read_csv("{}/y.dat".format(wds_data_folder), header=None, nrows=rows_cells * cols_cells,
                         delim_whitespace=True, dtype=np.float32)
y_original = y_original.values.flatten()

In [18]:
y_original

array([0., 0., 0., ..., 0., 0., 0.], dtype=float32)

In [11]:
print(x_original.shape)
print(y_original.shape)

(6400, 2)
(6400,)


In [12]:
# create a SVR object and specify the kernal and other parameters
svr_model = SVR(kernel='rbf', C=1.0, gamma=0.3, epsilon=.1)
# build the SVR power distribution model
svr_model.fit(x_original, y_original)

# save the SVR model to a file
pickle.dump(svr_model, open("{}/{}".format(wds_data_folder, svr_model_filename), 'wb'))

In [17]:
len(svr_model.support_vectors_)

5709

In [ ]:
del x_original
del y_original

In [7]:
svr_model=pickle.load(open("{}/{}".format(wds_data_folder, svr_model_filename), 'rb'))

In [19]:
##### SUGGA: support vector regression guided genetic algorithm
result_arr = np.zeros((num_runs, 2), dtype=np.float32)
for i in range(0, num_runs):  # run times
    print("run number: {} ...".format(i))
    wfg.load_init_pop_NA("{}/init_{}.dat".format(init_pops_data_folder, i),
                         "{}/init_{}_NA.dat".format(init_pops_data_folder, i))
    run_time, eta = wfg.sugga_genetic_alg(i, svr_model=svr_model, result_folder=sg_result_folder)
    result_arr[i, 0] = run_time
    result_arr[i, 1] = eta
time_stamp = datetime.now().strftime("%Y%m%d%H%M%S")
filename = "{}/result_sugga_{}.dat".format(sg_result_folder, time_stamp)
np.savetxt(filename, result_arr, fmt='%f', delimiter="  ")

run number: 0 ...


layouts:   0%|          | 0/10 [00:00<?, ?it/s]

Support vector regression guided genetic algorithm starts....
generation 0...


layouts: 100%|██████████| 10/10 [00:03<00:00,  3.30it/s]


[513.0141  512.8949  513.5583  514.7492  510.76782 512.9406  510.49908
 511.83102 513.9224  512.27686]


...

[514.01794 515.284   516.79205 516.99524 515.59454 515.4765  514.9277
 514.9772  515.7269  514.9612 ]


layouts:   0%|          | 0/10 [00:00<?, ?it/s]

generation 11...


layouts:  60%|██████    | 6/10 [00:02<00:01,  2.85it/s]


KeyboardInterrupt: 

### Shell Power Computation Package

In [1]:
import sys 
sys.path.append('../Shell_Code_Modified/')

In [2]:
import Farm_Evaluator_Vec

In [3]:
total_energy, turbine_power=Farm_Evaluator_Vec.get_AEP_results()

In [4]:
total_energy

array([1.2523587, 1.1595447, 1.1450489, 1.0768385, 1.176319 , 1.1840583,
       1.13568  , 1.079648 , 1.0935222, 1.1273198, 1.0859367, 1.167106 ,
       1.1016386, 1.124391 , 1.1225449, 1.100678 , 1.1647563, 1.2282459,
       1.1581823, 1.197542 , 1.2453799, 1.2442212, 1.2059841, 1.1259376,
       1.2380294, 1.1106503, 1.1325871, 1.1829029, 1.1153044, 1.1439447,
       1.1912581, 1.1264657, 1.1022447, 1.1789318, 1.1049944, 1.2037905,
       1.220672 , 1.1373823, 1.2131999, 1.1142861, 1.199062 , 1.141587 ,
       1.1335138, 1.1493688, 1.0722657, 1.0965657, 1.2305766, 1.2662541,
       1.1084578, 1.0826805], dtype=float32)

In [5]:
turbine_power

505.4506365966797